# Thermochronometry

The calculation below is based on the very last section of chapter 6 of Anderson and Anderson's *Geomorphology* book.
Briefly, many chemical reactions in rocks only occur above certain temperatures and pressures.
By examining the chemistry of rocks at earth's surface, one can ascertain how long that parcel of rock spent above a given temperature and pressure during its exhumation from the interior.
In the simplest settings, one assumes that the surfaces of constant temperature (isotherms) are flat, but in complex landscapes this is no longer true and one needs some estimate of the slope of the isotherms.
Several papers have explored using numerical models for this heat flow calculation and there are some open-source software packages for it as well:
* [Braun (2003)](https://doi.org/10.1016/S0098-3004(03)00052-9)
* [Braun (2005)](https://doi.org/10.2138/rmg.2005.58.13)
* [Ehlers (2005)](https://doi.org/10.2138/rmg.2005.58.12)
* [Ehlers and Farley (2003)](https://doi.org/10.1016/S0012-821X(02)01069-5)
* [Ehlers et al (2003)](https://doi.org/10.1029/2001JB001723)

The setup that this demo will reproduce is from figure 6.45 in Anderson and Anderson.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import firedrake
from firedrake import Constant, inner, grad, dx, ds

### Geometry

Here I'll show how to warp or deform an initial computational mesh into a new mesh.
First, we'll create an initial mesh of the unit square.
The colors in the plot below show the IDs of the boundary segments.
You'll need those later when we want to apply different boundary conditions.

In [ ]:
nx, nz = 32, 32
initial_mesh = firedrake.UnitSquareMesh(nx, nz, quadrilateral=True)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(initial_mesh, axes=ax)
ax.legend();

Now to create the jagged topography in figure 6.45 of A&A, we'll use the `firedrake.conditional` function.
The physical domain is 10km long, and there are break points at the positions $x$ = 0, 2.5, 5, 7.5, and 10km.
In our initial, computational domain, these will be at the positions 0, 1/4, 1/2, 3/4, and 1.
The depths at the break points in the figure are 0, 2, 0, 1, and 0km.
We'll reverse this and instead assume the highest peak is 2km above sea level.
The deepest point is at an elevation of -3km.

In [ ]:
xs = [Constant(x) for x in [0.0, 0.25, 0.5, 0.75, 1.0]]
hs = [Constant(h) for h in [2.0, 0.0, 2.0, 1.0, 2.0]]

We're going to start by making an expression for the surface elevation in the right-most segment of the domain.
The surface elevation linearly interpolates the values `hs[3]` and `hs[4]` between the points `xs[3]` and `xs[4]`.

In [ ]:
x, z = firedrake.SpatialCoordinate(initial_mesh)

ξ_4 = (x - xs[3]) / (xs[4] - xs[3])
expr4 = hs[3] * (1 - ξ_4) + hs[4] * ξ_4

But this expression alone doesn't describe the topography over the whole domain, only between the last two break points.
We can at least get something that's correct in the next interval to the left by using the conditional function.
When the condition `x <= xs[3]` evaluates to true, it picks the first expression, which now interpolates the height values between `xs[2]` and `xs[3]`; when it evaluates to false, it picks the second expression, which we just defined in the cell above.

In [ ]:
ξ_3 = (x - xs[2]) / (xs[3] - xs[2])
expr3 = firedrake.conditional(
    x <= xs[3],
    hs[2] * (1 - ξ_3) + hs[3] * ξ_3,
    expr4,
)

But why stop there?

In [ ]:
ξ_2 = (x - xs[1]) / (xs[2] - xs[1])
expr2 = firedrake.conditional(
    x <= xs[2],
    hs[1] * (1 - ξ_2) + hs[2] * ξ_2,
    expr3,
)

ξ_1 = (x - xs[0]) / (xs[1] - xs[0])
Z = firedrake.conditional(
    x <= xs[1],
    hs[0] * (1 - ξ_1) + hs[1] * ξ_1,
    expr2,
)

Finally, we'll make an expression for the mapping between the initial mesh and the computational mesh.
Since we're making a vector expression now, we pass a tuple or list of its components to the function `firedrake.as_vector`.
In the $x$ component, we linearly scale all the values up to the length of the domain, which is 10km.
In the $z$ component, we linearly interpolate between the elevation of the bottom of the domain (-3km) at the bottom of the computational domain, up to the expression for the surface elevation defined above at the top of the computational domain.

In [ ]:
Lx = Constant(10.0)
z_b = Constant(-3.0)
expr = firedrake.as_vector([Lx * x, (1 - z) * z_b + z * Z])

Finally, we extract the function space for the coordinates on the initial mesh, interpolate our incredibly annoying vector expression above, and define a new mesh based on these coordinates.

In [ ]:
Vc = initial_mesh.coordinates.function_space()
X = firedrake.Function(Vc).interpolate(expr)
mesh = firedrake.Mesh(X)

Now let's plot the result.
Note how the boundary labels stay the same.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_xlabel("x (km)")
ax.set_ylabel("z (km)")
firedrake.triplot(mesh, axes=ax)
ax.legend();

I've had a lot of practice at this.

### Modeling

Compared to that awful expression for the geometry, the modeling part will be easy.
We won't ever have to refer to the initial mesh or its coordinates again, so we can now make `x` and `z` stand for the spatial coordinates on our real mesh.

In [ ]:
x, z = firedrake.SpatialCoordinate(mesh)

First, we'll make the temperature field at the surface from a typical moist adiabatic lapse rate of 5${}^\circ$C / km.
We store the boundary IDs of the surface in the list `surf_ids`.
We can identify these by looking at the colors and legend in the figure above.

In [ ]:
T_sea_level = Constant(18.0)
Γ = Constant(5.0)
T_surf = T_sea_level - Γ * z
surf_ids = [4]

At the bottom of the domain, we'll assume a constant temperature of 90C.
This isn't what's actually used in the figure in A&A but good enough for demonstration purposes.

In [ ]:
T_bed = Constant(90.0)
bed_ids = [3]

Finally, we'll assume no heat flux through the side walls of the domain.
Google tells me that the thermal conductivity of rocks is somewhere between 0.4 and 7.0 W / m ${}^\circ$C.
I'll split the difference and take a thermal conductivity of 3.5 W / m ${}^\circ$C and convert it to km.

In [ ]:
k = Constant(3.5e3)

element = firedrake.FiniteElement("CG", "quadrilateral", 1)
V = firedrake.FunctionSpace(mesh, element)

T = firedrake.Function(V)
ϕ = firedrake.TestFunction(V)
F = k * inner(grad(T), grad(ϕ)) * dx

surf_bc = firedrake.DirichletBC(V, T_surf, surf_ids)
bed_bc = firedrake.DirichletBC(V, T_bed, bed_ids)
bcs = [surf_bc, bed_bc]

firedrake.solve(F == 0, T, bcs)

Now we'll plot the result.
I've added some contours every 10${}^\circ$C so you can see the bending of the isotherms.
As A&A note in the book chapter, the isotherms are basically flat below a depth equal to the landscape relief.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_xlabel("x (km)")
ax.set_ylabel("z (km)")
colors = firedrake.tripcolor(T, cmap="inferno", axes=ax)
fig.colorbar(colors, label="temperature (${}^\circ$C)")
levels = np.linspace(0.0, 80.0, 9)
firedrake.tricontour(T, levels=levels, colors="grey", axes=ax);

### Now what

Another way that we could have done our computations would be to do our actual computing on the initial mesh, but reformulate everything in terrain-following coordinates.
This would give us a reasonably simple way to deal with variable surface topography, from say erosion or other surface processes.

Other things to try:
* add upward advection of heat or exhumation
* use a fixed heat flux from below the crust; figure out a sensible value that roughly gives the same slope as above
* use a more realistic (variable) thermal conductivity
* simulate a chemical reaction occuring along with the exhumation; make it temperature-dependent